#### IMPORTS

In [ ]:
import pandas as pd
import numpy as np
import re


#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df = pd.read_csv('../files/hr_raw_data.csv', index_col=0)


#### VARIABLES AND FUNCCIONS

In [ ]:
#COLUMN RENAMES:

title_mapping = {"employeenumber": "employee_number",
                "gender": "gender",
                "datebirth": "birth_year",
                "age": "age",
                "maritalstatus": "marital_status",
                "jobrole": "job_title",
                "department": "department",
                "attrition": "terminated",
                "standardhours": "standard_hours",
                "monthlyincome": "monthly_income",
                "remotework": "remote",
                "businesstravel": "business_travel",
                "dailyrate": "daily_rate",
                "distancefromhome": "dist_home",
                "education": "education_scale",
                "educationfield": "education",
                "environmentsatisfaction": "env_sat_rate",
                "hourlyrate": "hourly_rate",
                "jobinvolvement": "job_involvement",
                "joblevel": "job_level",
                "jobsatisfaction": "job_sat_rate",
                "monthlyrate": "monthly_rate",
                "numcompaniesworked": "num_comp_worked",
                "overtime": "over_time",
                "percentsalaryhike": "perc_salary_hike",
                "performancerating": "perf_rate",
                "relationshipsatisfaction": "relationship_sat_rate",
                "stockoptionlevel": "stock_opt_level",
                "totalworkingyears": "tot_working_year",
                "trainingtimeslastyear": "traning_times_last_year",
                "worklifebalance": "work_life_balance",
                "yearsatcompany": "year_at_comp",
                "yearsincurrentrole": "year_current_role",
                "yearssincelastpromotion": "year_last_promotion",
                "yearswithcurrmanager": "year_current_mngr",
                "salary": "annual_salary",
                "roledepartament": "role_department"}

#CATEGORIES: 

columns_personal =  ['employee_number', 
                    'gender', 
                    'birth_year', 
                    'age', 
                    'marital_status',
                    'dist_home']

columns_job =   ['job_title',
                 'role_1',
                 'department_1',
                'department',
                'terminated',
                'year_at_comp',
                'year_current_role',
                'standard_hours',
                'remote',
                'business_travel',
                'over_time', 
                'job_level', 
                'stock_opt_level', 
                'traning_times_last_year', 
                'perf_rate',
                'year_last_promotion',
                'year_current_mngr']

columns_education = ['education',
                    'education_scale']


columns_income =    ['annual_salary',
                    'monthly_income',
                    'daily_rate',
                    'hourly_rate',
                    'monthly_rate',
                    'perc_salary_hike']

columns_satisfaction =  ['env_sat_rate',
                        'job_involvement',
                        'job_sat_rate',
                        'relationship_sat_rate',
                        'work_life_balance']

columns_emp_bgd =   ['num_comp_worked',
                    'tot_working_year']


#COLUMN REORDER:

new_order_columns = columns_personal+columns_job+columns_education+columns_income+columns_satisfaction+columns_emp_bgd

def reorder_columns(df, list_columns):

    #Reorder columns as per provided list, add all the missing ones at the end.
    
    try:
        df = df[new_order_columns]
        extra_columns = [col for col in df.columns if col not in new_order_columns]
        df = df[ new_order_columns + extra_columns ]
    
    except KeyError as e:
        print(f"KeyError: {e}")
        missing_columns = [col for col in new_order_columns if col not in df.columns]
        print(f"Missing columns: {missing_columns}")

        extra_columns = [col for col in df.columns if col not in new_order_columns]
        df = df[ new_order_columns + extra_columns ]
    
    return df


#DATA ANALYSIS

def data_analysis (dataframe, columns):

    print('📌CATEGORY ANALYSIS:\n')
    display(df[columns_personal].describe().T)
    print('\n')

    for col in columns:
        
        print(f'📌Columna: {col.upper()} \n')
        print('Null counter:', dataframe[col].isnull().sum(), '\n')
        print('These are the unique values:', dataframe[col].unique(),'\n')
        print('These are values counter:', dataframe[col].value_counts(dropna=False),'\n')
        print('-------------')


#### STRUCTURE CLEANING

In [ ]:
#Splitting column "roledepartament"
# df[["role_1", "department_1"]]=df["roledepartament"].str.split("-", expand=True).get([0, 1])

In [ ]:
#RENAME
df = df.rename(columns=title_mapping)

In [ ]:
#REORDER
df = reorder_columns(df, new_order_columns)

In [ ]:
df.head()

In [ ]:
# Title and strip values

to_title = ['marital_status', 'job_title', 'role_1', 'department_1', 'education']

df[to_title] = df[to_title].apply(lambda x: x.str.title().str.strip() if x.dtype == 'object' else x)

In [ ]:
# Update 0/1 from Genger to M/F

gender_map = {0: "M", 1:"F"}
df['gender'] = df['gender'].map(gender_map)

In [ ]:
# Replacing values obj to numeric in age

age_update = {
    'thirty-two': 32,
    'twenty-four': 24,
    'thirty': 30,
    'fifty-eight': 58,
    'fifty-two': 52,
    'twenty-six': 26,
    'fifty-five': 55,
    'thirty-seven': 37,
    'thirty-six': 36,
    'forty-seven': 47,
    'thirty-one': 31}

df['age'] = df['age'].replace(age_update)


In [ ]:
def convert_to_float(df, value):
    try:
        value = value.replace(",", ".")
        value = value.replace("$", "")
        return float(value)
    
    except: return value
    

In [3]:
to_floats = ['annual_salary', 'monthly_income', 'daily_rate','hourly_rate']
df[to_floats] = df[to_floats].apply(convert_to_float)




NameError: name 'df' is not defined

#### DATA ANALYSYS

##### PERSONAL COLUMNS

In [ ]:
data_analysis(df, columns_personal)

##### JOB COLUMNS

In [ ]:
data_analysis(df, columns_job)

##### INCOME COLUMNS

In [ ]:
data_analysis(df, columns_income)

##### EDUCATION COLUMNS

In [ ]:
data_analysis(df,columns_education)

##### SATISFACTION COLUMNS

In [ ]:
data_analysis(df,columns_satisfaction)

##### EMPLOYEMENT BACKGROUND COLUMNS

In [ ]:
data_analysis(df, columns_emp_bgd)

In [ ]:
df.head()